
### 1.- Diccionario de atributos
anime_id - Identificador de anime de la pagina myanimelist.net's, es unico para cada anime.

name - Nombre Completo del anime.

genre - Genero del anime.

type - Serie, pelicula, Ova etc.

episodes - Cuantos Episodios tiene.

rating - Puntuación del anime.

members - Numero de miembros en su grupo de fans.

user_id - Codigo de identificación de usuario, se genera aleatoreamente.

rating - Puntuación de 1 a 10. El valor -1 denota animes vistos por el usuario pero no puntuados.




### 2.- Importación de Librerias

In [ ]:
import dask.dataframe as dd
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise import SlopeOne
from surprise import CoClustering
from surprise import BaselineOnly
from surprise.accuracy import rmse
from surprise import accuracy
from surprise.model_selection import train_test_split
from collections import defaultdict

### 3.- Importación de Datos

In [ ]:
ratings = dd.read_csv('../input/anime-recommendations-database/rating.csv')
ratings.head(2)

In [ ]:
#Para evitar confusiones con los nombres
ratings=ratings.rename(columns = {'rating':'user_rating'})

In [ ]:
#importamos la data de anime
anime = dd.read_csv('../input/anime-recommendations-database/anime.csv')
anime.head(2)

### 3.1- Exploración de datos

In [ ]:
#Numero de usuarios unicos 
ratings.user_id.nunique().compute()

In [ ]:
#información basica sobre sobre ratings
sns.distplot(ratings.user_rating)
ratings.describe().compute()

In [ ]:
users=ratings.groupby('user_id').agg({'user_rating':['mean','count','std']})
users.columns=['media','n_puntuaciones','std_puntuaciones']

In [ ]:
sns.distplot(users.media,kde=False)
users.media.describe().compute()

In [ ]:
sns.distplot(users.n_puntuaciones)
users.n_puntuaciones.describe().compute()

### 3.2.- Conclusiones de la Exploración
La media de las puntuaciones de los usuarios es 8.2277 con una aumento en la puntuación 10. A priori se esperaría puntuaciones más centrado en 5, que no sea así puede explicarse suponiendo el sesgo de que esta base esté hecha con personas a las que en general simplemente les gusta el anime. Sabemos que las puntuaciones se realizan en numeros enteros, por lo tanto podemos decir que puntuaciones menores qué 8 significarían algo como "me gustó pero no tanto", mientras que las superiores a 8 son verdaderas medidas de que les gusto, a la vez que la puntuación 8 significaría algo como "me gustó lo mismo que la mayoría".

### 4.- Selección de datos utiles
Nos interesan los usuarios que hayan evaluado almenos 1 anime, y sin considerar las valoraciones -1 ya que no aportan información a los ratings. Además tampoco nos sirven usuarios con muy pocas valoraciones.

In [ ]:
# Esto significa:
sin_puntuaciones=ratings.groupby(['user_id']).agg({'user_rating':['count','sum']})
print (len(sin_puntuaciones[-sin_puntuaciones['user_rating','sum']==sin_puntuaciones['user_rating','count']]),' usuarios perdidos')

In [ ]:
#Nos deshacemos de ambos
ratings=ratings[ratings.user_rating>0]

Tampoco nos sirven usuarios con muy pocas valoraciones, no nos gustaría perder más de un 10%


In [ ]:
users=ratings.groupby('user_id').agg({'user_rating':['mean','count','std']})
users.columns=['MPRU','n_puntuaciones','std_puntuaciones']

In [ ]:
# %  de perdida de usuarios con solo una puntuación:
100*len(users[users.n_puntuaciones<=1])/len(users)

In [ ]:
# % de perdida de usuarios con 2 o menos puntuaciones:
100*len(users[users.n_puntuaciones<=2])/len(users)

In [ ]:
# % de perdida de usuarios con 3 o menos puntuaciones:
100*len(users[users.n_puntuaciones<=3])/len(users)

#### Un 10.22% nos parece aceptable, por lo que usaremos usuarios con 4 o más puntuaciones.


In [ ]:
#usuarios con 4 o más puntuaciones
users=users[users.n_puntuaciones>3]

In [ ]:
#Juntamos los datasets para tener la informacipon completa
data=dd.merge(ratings,users,on='user_id',how='inner')

In [ ]:
df=dd.merge(data[['user_id','anime_id','user_rating']],anime[['anime_id','name']],on='anime_id',how='left',indicator=True)


In [ ]:
df.head()

#### Es posible que algunos datos no nos hayam cruzado, revisemos.


In [ ]:
#Revisemos Cuantos no nos cruzaron.
len(data)-len(df[df._merge=='both'])

In [ ]:
#Esto significa que hay animes id sin nombre correspondiente.
df.anime_id[df['_merge']=='left_only'].compute().unique()

#### El anime_id 30913 no tiene un nombre correspondiente, y ha sido puntuado en dos ocasiones.



In [ ]:
#dejamos fuera aquellas filas cuyo anime_id no se corresponde entre los dos sets.
df=df[df._merge=='both']

In [ ]:
df_triple=df[['user_id','anime_id','user_rating']]

In [ ]:
len(df_triple)

In [ ]:
reader = Reader(rating_scale=(1.0, 10.0))
data = Dataset.load_from_df(df_triple[['user_id', 'anime_id', 'user_rating']], reader)

### 5.- Escogiendo el Mejor Algoritmo
Probaremos 3 algoritmos que por lo general dan buenos resultados, todos disponibles en la libreria "Surprise".

### BaselineOnly
Este algoritmo basico predice de la siguiente forma:

$$b_{ui} = \mu + b_{u} + b_{i}$$
donde $b_{ui}$ es la puntuación desconocida, $\mu$ la puntuación media del anime, $b_{u}$ es la desviación de puntuaciones del usuario, y $b_{i}$ la desviación de este anime con respecto a la media del total de puntuaciones.

Por ejemplo, supongamos que queremos una estimación de la calificación de Fullmetal Alchemist Brotherhood por un usuario X digamos que la calificación promedio de todas los animes $\mu$, es de 8.2 Además, FMA Brotherhood es mejor que un anime promedio, por lo que digamos que tiene una puntuación 1.5 por encima del promedio ( $b_{i}$ ). Por otro lado, X es un usuario exigente que tiende a calificar 0.3 más bajo que el promedio ( $b_{u}$ ). Por lo tanto, la estimación de Baseline sería:$$ 8.2-0.3+1.5=9.4$$

http://courses.ischool.berkeley.edu/i290-dm/s11/SECURE/a1-koren.pdf

https://surprise.readthedocs.io/en/stable/basic_algorithms.html

In [ ]:
#El performance del algoritmo estara dado por una validación cruzada de 5 iteraciones.
#El conjunto de testeo es una valoración por usuario en el dataset.
results = cross_validate(BaselineOnly(), data, measures=['RMSE'], cv=5, verbose=False)
basel = pd.DataFrame.from_dict(results).mean(axis=0)
basel = basel.append(pd.Series([str(BaselineOnly()).split(' ')[0].split('.')[-1]], index=['BaselineOnly()']))

In [ ]:
basel

### Co-Clustering
Tanto a los usuarios como a los elementos se les asignan clusters $C_u$ y $C_i$ respectivamente, y algunos co-clusters $C_{ui}$.

La predicción se hace de la siguiente forma:$$\hat{r}_{ui} = \overline{C_{ui}} + (\mu_u - \overline{C_u}) + (\mu_i- \overline{C_i})$$donde $\overline{C_{ui}}$ es la calificación promedio de los co-clusters. $\overline{C_u}$ es La calificación promedio de los clusters de usuarios está denotada y $\overline{C_u}$ es la calificación del cluster de elementos (items). $\mu_u$ es la media de la puntuación del usuario u, y $\mu_i$ es la media de puntuaciones dado al item i.

https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.113.6458&rep=rep1&type=pdf

https://surprise.readthedocs.io/en/stable/notation_standards.html#george-2005

In [ ]:
#El performance del algoritmo estara dado por una validación cruzada de 5 iteraciones.
#El conjunto de testeo es una valoración por usuario en el dataset.
results = cross_validate(CoClustering(), data, measures=['RMSE'], cv=5, verbose=False)
coclus = pd.DataFrame.from_dict(results).mean(axis=0)
coclus = coclus.append(pd.Series([str(CoClustering()).split(' ')[0].split('.')[-1]], index=['CoClustering()']))

In [ ]:
coclus

### SlopeOne
Sus predicciones se definen como: $$ \hat{r}_{ui} = \mu_u + \frac{1}{|R_i(u)|}\sum\limits_{j \in R_i(u)} \text{dev}(i, j)$$ donde$R_i(u)$ es el conjunto de elementos relevantes, es decir, el conjunto de elementos j valorados por por el usuario u que también tienen al menos un usuario común con i. Además $\text{dev}(i, j)$ se define como la diferencia promedio entre las calificaciones de i y las de j.

"Los algoritmos de Slope One son fáciles de implementar, eficientes para consultar, razonablemente precisos y admiten consultas en línea y actualizaciones dinámicas, lo que los convierte en buenos candidatos para sistemas del mundo real. El esquema básico de pendiente uno se sugiere como un nuevo esquema de referencia para el filtrado colaborativo. Al tener en cuenta los elementos que a un usuario le gustaron por separado de los elementos que no le gustaron, logramos resultados competitivos..."

https://arxiv.org/abs/cs/0702144

https://surprise.readthedocs.io/en/stable/slope_one.html#surprise.prediction_algorithms.slope_one.SlopeOne

In [ ]:
#El performance del algoritmo estara dado por una validación cruzada de 5 iteraciones.
#El conjunto de testeo es una valoración por usuario en el dataset.
results = cross_validate(SlopeOne(), data, measures=['RMSE'], cv=5, verbose=False)
slope = pd.DataFrame.from_dict(results).mean(axis=0)
slope = slope.append(pd.Series([str(SlopeOne()).split(' ')[0].split('.')[-1]], index=['SlopeOne()']))

In [ ]:
slope

El algoritmo con mejor desempeño en terminos de error cuadratico medio del conjunto de testeo es SlopeOne, veamos como se comporta con un set de testeo del 20%.

In [ ]:
trainset, testset = train_test_split(data, test_size=0.20)
slopeone=SlopeOne()
predict_test = slopeone.fit(trainset).test(testset)
accuracy.rmse(predict_test)

### Hagamos un top por usuario con los resultados

In [ ]:
def top(user,predict_test=predict_test,anime=anime):
    anime=anime[['anime_id','name']]
    predichos=dd.from_pandas(pd.DataFrame.from_records(predict_test), npartitions=6)
    predichos=predichos[[0,1,2,3]]
    predichos.columns=['user_id','anime_id','p_real','p_predicha']
    predichos=predichos[predichos.user_id==user]
    predichos.p_predicha=predichos.p_predicha.round(decimals=0)
    predichos=predichos.compute()
    #Ya lo redujimos lo suficiente como para usar pandas con normalidad.
    predichos=pd.DataFrame(predichos.sort_values(by='p_predicha',ascending=False).head(3))
    return(pd.merge(predichos,anime.compute(),on='anime_id',how='inner'))

In [ ]:
predichos=dd.from_pandas(pd.DataFrame.from_records(predict_test), npartitions=6)
predichos=predichos[[0,1,2,3]]
predichos.columns=['user_id','anime_id','p_real','p_predicha']

**Juguemos un poco ahora**
p_real - Es la puntuación que realmente dieron, por supuesto el algoritmo no tomo en cuenta estas puntuaciones para su predicción.

p_predicha - Es la puntuación predicha por el algoritmo.[](http://)

In [ ]:
top(33888)

In [ ]:
top(28859)

In [ ]:
top(5255)

In [ ]:
top(65451)